In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from config import db_password
from flask import Flask
from flask_sqlalchemy import sqlalchemy
import os
import re
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
path = 'Resources/'
degrees = pd.read_csv(path +'degrees_clean.csv', index_col=0)
degrees

,object_id,degree_type,subject,institution,graduated_at
0,p:6117,MBA,NaN,NaN,NaN
1,p:6136,BA,"English, French","Washington University, St. Louis",1/1/1990
2,p:6136,MS,Mass Communication,Boston University,1/1/1992
3,p:6005,MS,Internet Technology,University of Greenwich,1/1/2006
4,p:5832,BCS,"Computer Science, Psychology",Rice University,NaN
...,...,...,...,...,...
109605,p:268589,CPA,NaN,American Institute of Certtified Public Accoun...,NaN
109606,p:268527,MS & BS,Engineering,South China University of Technology,NaN
109607,p:268527,PhD,Engineering,Clarkson University,NaN
109608,p:268528,B.S.,Electrical Engineering,Colorado State University,NaN


In [11]:
degrees_red = degrees[['object_id', 'degree_type']]
degrees_red

,object_id,degree_type
0,p:6117,MBA
1,p:6136,BA
2,p:6136,MS
3,p:6005,MS
4,p:5832,BCS
...,...,...
109605,p:268589,CPA
109606,p:268527,MS & BS
109607,p:268527,PhD
109608,p:268528,B.S.


In [3]:
objects = pd.read_csv(path +'objects_clean.csv', index_col=0)
objects

C:\Users\phsstudent\anaconda3\envs\mlenv\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (3,5,7,8,9,10,11,13,14,17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,object_id,entity_type,parent_id,name,category_code,status,founded_at,closed_at,country_code,state_code,city,region,first_investment_at,last_investment_at,investment_rounds,invested_companies,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
0,c:1,Company,NaN,Wetpaint,web,operating,2005-10-17,NaN,USA,WA,Seattle,Seattle,NaN,NaN,0,0,2005-10-01,2008-05-19,3,39750000.0,2010-09-05,2013-09-18,5,17
1,c:10,Company,NaN,Flektor,games_video,acquired,NaN,NaN,USA,CA,Culver City,Los Angeles,NaN,NaN,0,0,NaN,NaN,0,0.0,NaN,NaN,0,6
2,c:100,Company,NaN,There,games_video,acquired,NaN,NaN,USA,CA,San Mateo,SF Bay,NaN,NaN,0,0,NaN,NaN,0,0.0,2003-02-01,2011-09-23,4,12
3,c:10000,Company,NaN,MYWEBBO,network_hosting,operating,2008-07-26,NaN,NaN,NaN,NaN,unknown,NaN,NaN,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0
4,c:10001,Company,NaN,THE Movie Streamer,games_video,operating,2008-07-26,NaN,NaN,NaN,NaN,unknown,NaN,NaN,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462646,r:9995,Product,c:14164,"SiteLink, listing feed for Brokerages",NaN,operating,NaN,NaN,NaN,NaN,NaN,unknown,NaN,NaN,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0
462647,r:9996,Product,c:14164,"EDCLink, listing feed for Economic Development...",NaN,operating,NaN,NaN,NaN,NaN,NaN,unknown,NaN,NaN,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0
462648,r:9997,Product,c:14164,"Cmail, broadcast email marketing",NaN,operating,NaN,NaN,NaN,NaN,NaN,unknown,NaN,NaN,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0
462649,r:9998,Product,c:14164,"CatylistCRM, contact database",NaN,operating,NaN,NaN,NaN,NaN,NaN,unknown,NaN,NaN,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0


In [6]:
obj_people = objects[objects['object_id'].str.contains('p:', regex=True)]

In [15]:
obj_people_red = obj_people[['object_id', 'entity_type', 'name']]

In [25]:
combined = obj_people_red.merge(degrees_red, how='outer')

In [30]:
combined[combined['name'].isnull()]

,object_id,entity_type,name,degree_type
267693,p:6611,NaN,NaN,BS
267694,p:6614,NaN,NaN,BS
267695,p:6614,NaN,NaN,BS
267696,p:6659,NaN,NaN,BS
267697,p:6729,NaN,NaN,BA (Hons) First Class
...,...,...,...,...
268241,p:20063,NaN,NaN,NaN
268242,p:2739,NaN,NaN,Msc
268243,p:2739,NaN,NaN,Bsc
268244,p:2739,NaN,NaN,PG


In [43]:
degree_clean = degrees[degrees['object_id'].isin(objects['object_id'])]

In [44]:
degree_clean.to_csv(path + 'degrees_clean.csv')

In [45]:
people = pd.read_csv(path +'people_clean.csv', index_col=0)
people

,object_id,first_name,last_name,affiliation_name
0,p:2,Ben,Elowitz,Blue Nile
1,p:3,Kevin,Flaherty,Wetpaint
2,p:4,Raju,Vegesna,Zoho
3,p:5,Ian,Wenig,Zoho
4,p:6,Kevin,Rose,i/o Ventures
...,...,...,...,...
226703,p:268587,Mark,Stolper,Unaffiliated
226704,p:268589,John,Pins,Unaffiliated
226705,p:268590,David,Schulhof,Unaffiliated
226706,p:268592,Matthew,D. Rosen,Unaffiliated


In [41]:
people = pd.read_csv(path +'people_clean.csv', index_col=0)
people

,object_id,first_name,last_name,affiliation_name
0,p:2,Ben,Elowitz,Blue Nile
1,p:3,Kevin,Flaherty,Wetpaint
2,p:4,Raju,Vegesna,Zoho
3,p:5,Ian,Wenig,Zoho
4,p:6,Kevin,Rose,i/o Ventures
...,...,...,...,...
226703,p:268587,Mark,Stolper,Unaffiliated
226704,p:268589,John,Pins,Unaffiliated
226705,p:268590,David,Schulhof,Unaffiliated
226706,p:268592,Matthew,D. Rosen,Unaffiliated


In [46]:
people_clean = people[people['object_id'].isin(objects['object_id'])]
people_clean

,object_id,first_name,last_name,affiliation_name
0,p:2,Ben,Elowitz,Blue Nile
1,p:3,Kevin,Flaherty,Wetpaint
2,p:4,Raju,Vegesna,Zoho
3,p:5,Ian,Wenig,Zoho
4,p:6,Kevin,Rose,i/o Ventures
...,...,...,...,...
226703,p:268587,Mark,Stolper,Unaffiliated
226704,p:268589,John,Pins,Unaffiliated
226705,p:268590,David,Schulhof,Unaffiliated
226706,p:268592,Matthew,D. Rosen,Unaffiliated
